1. Mount Google drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


2. Change Runtime to "GPU"

3. Test a sample using the pre-trained ResNet101

In [4]:
import torch, torchvision
from PIL import Image

# load model
resnet = torchvision.models.resnet101(pretrained=True)

# set network to evaluation mode
resnet.eval()

transform = torchvision.transforms.Compose([          
 torchvision.transforms.Resize(256),                   
 torchvision.transforms.CenterCrop(224),               
 torchvision.transforms.ToTensor(),                     
 torchvision.transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                            
 std=[0.229, 0.224, 0.225]                             
 )])


img = Image.open("/content/gdrive/MyDrive/SUTD_GSUITE/Year 3/Term 7/DeepLearning/Hw3/dog.jpg") # You can download an image of a dog from Internet or capture an image by yourself.
img_t = transform(img)
print(img_t.shape)

batch_t = torch.unsqueeze(img_t, 0)


# perform inference
out = resnet(batch_t)

# print top-5 classes predicted by model
_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
for idx in indices[0][:5]:
  print('Label:', idx, '. Confidence Score:', percentage[idx].item(), '%')


torch.Size([3, 224, 224])
Label: tensor(259) . Confidence Score: 64.96243286132812 %
Label: tensor(153) . Confidence Score: 18.492586135864258 %
Label: tensor(258) . Confidence Score: 3.901040554046631 %
Label: tensor(152) . Confidence Score: 3.283771514892578 %
Label: tensor(265) . Confidence Score: 1.451918601989746 %


4. Refer to https://gist.github.com/ageitgey/4e1342c10a71981d0b491e1b8227328b, to check if the predicted classes are meaningful.

5. Task

Modify the code above, to peform data augmentation for the testing sample (averaging the scores of 5 crops: center crop, upper left crop, lower left crop, lower right crop, upper right crop).

Pls briefly discuss the advantages and disadvantages of using testing data augmentation.

In [3]:
import torch, torchvision
from PIL import Image
from torchvision import transforms

def perRestNet(img_t):
    batch_t = torch.unsqueeze(img_t, 0)

    # perform inference
    out = resnet(batch_t)
    return out


# load model
resnet = torchvision.models.resnet101(pretrained=True)

# set network to evaluation mode
resnet.eval()

img = Image.open("/content/gdrive/MyDrive/SUTD_GSUITE/Year 3/Term 7/DeepLearning/Hw3/dog.jpg") # You can download an image of a dog from Internet or capture an image by yourself.


########### Not allowed to use inbuild function, but it works for references ############

# transform2 = torchvision.transforms.Compose([transforms.Resize(256),
#                             transforms.FiveCrop(224), # this is a list of PIL Images
#                             transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
#                             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#                            ])

# img_t = transform2(img)
# print(img_t.shape)

# # perform inference
# out = resnet(img_t)

# # print top-5 classes predicted by model
# _, indices = torch.sort(out, descending=True)
# percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
# for idx in indices[0][:5]:
#   print('Label:', idx, '. Confidence Score:', percentage[idx].item(), '%')

transform3 = torchvision.transforms.Compose([       
 torchvision.transforms.Resize(256),                   
 torchvision.transforms.ToTensor(),                     
 torchvision.transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                            
 std=[0.229, 0.224, 0.225]                             
 )])

img_t = transform3(img)
scores_sum = 0
outputTorch = torch.zeros((1,1000))

for i in range(0,5):
    slicedIMG = img_t
    if(i==0):
        slicedIMG = img_t[:,0:224,0:224] ## torch.Size([3, 224, 224])
    if(i==1):
        slicedIMG = img_t[:,0:224,img_t.shape[2]-224:img_t.shape[2]]  ## torch.Size([3, 224, 224])
    if(i==2):
        slicedIMG = img_t[:,img_t.shape[1]-224:img_t.shape[1],0:224]
    if(i==3):
        slicedIMG = img_t[:,img_t.shape[1]-224:img_t.shape[1],img_t.shape[2]-224:img_t.shape[2]]
    if(i==4):
        slicedIMG = img_t[:,round(img_t.shape[1]/2-112):round(img_t.shape[1]/2+112),round(img_t.shape[2]/2-112):round(img_t.shape[2]/2+112)]
        
    outputTorch = outputTorch.add(perRestNet(slicedIMG))

outputTorch = torch.div(outputTorch,5)

print("Averaging scores of 5 crops:")
# print top-5 classes predicted by model
_, indices = torch.sort(outputTorch, descending=True)
percentage = torch.nn.functional.softmax(outputTorch, dim=1)[0] * 100
for idx in indices[0][:5]:
    print('Label:', idx, '. Confidence Score:', percentage[idx].item(), '%')

Averaging scores of 5 crops:
Label: tensor(259) . Confidence Score: 67.27539825439453 %
Label: tensor(153) . Confidence Score: 16.193790435791016 %
Label: tensor(258) . Confidence Score: 5.253355026245117 %
Label: tensor(265) . Confidence Score: 2.458286762237549 %
Label: tensor(152) . Confidence Score: 1.5673789978027344 %
